# Decision Tree Classifier for a plagiarism detector

## Building the data dataframe

In [1]:
# Load modules and packages
import pandas as pd
import numpy as np

df = pd.read_csv("../data/train_scores.csv")

print(f"Plagiarized = {len(df[df['plagiarized'] == 1])}")
print(f"Non-plagiarized = {len(df[df['plagiarized'] == 0])}")
df.head()

Plagiarized = 685
Non-plagiarized = 331315


,src,sus,3-tok-lem-ngram,3-tok-lower-stop-alpha-ngram,3-tok-ngram,plagiarized
0,source-document00094.txt,suspicious-document00019.txt,0.000000,0.0,0.000000,0
1,source-document00029.txt,suspicious-document00019.txt,0.003139,0.0,0.002788,0
2,source-document00095.txt,suspicious-document00019.txt,0.003399,0.0,0.003399,0
3,source-document00081.txt,suspicious-document00019.txt,0.001540,0.0,0.001368,0
4,source-document00005.txt,suspicious-document00019.txt,0.001892,0.0,0.001734,0


# Training a tree classificator

In [2]:
from bokeh.plotting import *

# Tell bokeh where to output
output_notebook()

Loading BokehJS ...

Tenemos un problema con los datos, por la propia naturaleza del problema, hay muchos más casos de documentos original que de plagiados. Si no lo tenemeos en cuenta, dado que el calisificador busca el mayor acierto posible tomará la ruta más fácil que es decir siempre que no se trata de un documento plagiado. Dado que se trata de un prueba de concepto, simplemente recortamos nuestra muestre e introducimos en los datos para el entrenamiento y validación la misma cantidad de entradas de documentos original como plagiados.

Otra solución que se podría explorar es la asignación de pesos a los datos.

In [3]:
# Balance dataframe by undersampling, we assume naively that they should be in equal
n = len(df[df['plagiarized'] == 1])
data = df[df['plagiarized'] == 1].append(df[df['plagiarized'] == 0].sample(n = n))
data.reset_index(inplace = True, drop = True)

# Select data to model
#features = ["jaccard", "containment", "dep"]
X = data.drop(["src", "sus", "plagiarized"], axis=1, errors="ignore")
y = data["plagiarized"]

A continuación comprobaremos como de bien se desenvuelve un clasificador base con nuestros datos. Para medir esto usaremos el puntuador F1, ya que nos da una buena idea de la precision a la hora de clasificar cada tipo de documento. Para ello, usaremos K-Folds, ya que no tenemos tantos datos como para poder dividirls directamente en test y validación

In [12]:
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import KFold
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
import numpy as np

def eval_model_table(X, y, clf):
    # Make folds of data for cross validation
    k_fold = KFold(n_splits = 5, shuffle = True)

    results = []
    for train, test in k_fold.split(X):
        clf.fit(X.iloc[train], y.iloc[train])
        sc = score(y.iloc[test], clf.predict(X.iloc[test]), average = None, labels = [0, 1])
        results.append(np.stack(sc, axis = 0))

    avg_results = sum(results) / len(results)
    std_results = [np.abs(x - avg_results)**2 for x in results]
    std_results = np.sqrt(sum(std_results) / (len(std_results) * np.sqrt(len(std_results))))
    
    # Calculation of scores
    scores = pd.DataFrame(np.concatenate((avg_results, std_results), axis = 1))
    scores.columns = ["Avg. non-plagiarized", "Avg. plagiarized", "Std. non-plagiarized", "Std. plagiarized"]
    scores.insert(0, "Score type", ["precision", "recall", "F1", "support"])
    scores.set_index("Score type", inplace = True)
    
    return(scores)

# Init classifier, maybe SV could be useful?
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
eval_model_table(X, y, clf)

,Avg. non-plagiarized,Avg. plagiarized,Std. non-plagiarized,Std. plagiarized
Score type,,,,
precision,0.673279,0.904592,0.018095,0.039524
recall,0.941422,0.542130,0.026490,0.032830
F1,0.784051,0.675200,0.009952,0.023429
support,137.000000,137.000000,3.806537,3.806537


## Optimización de la profundidad máxima del arbol

In [23]:
# Lets try to optimize max_depth
from sklearn.metrics import f1_score
from statsmodels.nonparametric.smoothers_lowess import lowess


def calc_F1_depth(c):
    clf = make_pipeline(StandardScaler(), SVC(gamma='auto', C = c))

    # Make folds of data for cross validation
    k_fold = KFold(n_splits = 5, shuffle = True)

    results = []
    for train, test in k_fold.split(X):
        clf.fit(X.iloc[train], y.iloc[train])
        results.append(f1_score(y.iloc[test], clf.predict(X.iloc[test]), average = None))
    
    return sum(results) / len(results)
        
depth = np.arange(0.1, 500, 1)
F1 = np.stack(list(map(calc_F1_depth, depth.tolist())), axis = 0)

depth_plot = figure(title = "F1 score as a function of the maximum depth of the tree",
                   x_axis_label = "Depth",
                   y_axis_label = "F1",
                   sizing_mode = "stretch_width")

depth_plot.line(depth, F1[:, 0], legend_label = "Non-plagiarized", line_color = "steelblue")
depth_plot.line(depth, F1[:, 1], legend_label = "Plagiarized", line_color = "orange")
show(depth_plot)


In [27]:
# Lets try to optimize max_depth
from sklearn.metrics import f1_score
from statsmodels.nonparametric.smoothers_lowess import lowess


def calc_F1_depth(g):
    clf = make_pipeline(StandardScaler(), SVC(gamma=g))

    # Make folds of data for cross validation
    k_fold = KFold(n_splits = 5, shuffle = True)

    results = []
    for train, test in k_fold.split(X):
        clf.fit(X.iloc[train], y.iloc[train])
        results.append(f1_score(y.iloc[test], clf.predict(X.iloc[test]), average = None))
    
    return sum(results) / len(results)
        
depth = np.arange(0.01, 1, 0.01)
F1 = np.stack(list(map(calc_F1_depth, depth.tolist())), axis = 0)

depth_plot = figure(title = "F1 score as a function of the maximum depth of the tree",
                   x_axis_label = "Depth",
                   y_axis_label = "F1",
                   sizing_mode = "stretch_width")

depth_plot.line(depth, F1[:, 0], legend_label = "Non-plagiarized", line_color = "steelblue")
depth_plot.line(depth, F1[:, 1], legend_label = "Plagiarized", line_color = "orange")
show(depth_plot)

Como podemos ver, aunque localmente el resultado varie considerablemente, llega un momento donde oscila en torno al mismo valor. Dejaremos que el propio clasificador use el numero de niveles que la haga falta.

## Optimización de la proporcion de datos

Como hemos dicho antes, nuestros datos no estan balanceados. Lo que haremos a continuación es estudiar el impacto de tener más o menos datos de casos que no son plagia en relación a los que si lo son.

In [6]:
# Lets try to optmize by changing proportion of data
def calc_F1_prop(p):
    n = len(df[df['plagiarized'] == 1])
    m = round(p * n)
    data = df[df['plagiarized'] == 1].append(df[df['plagiarized'] == 0].sample(n = m))
    data.reset_index(inplace = True)
    
    X = data.drop(["src", "sus", "plagiarized"], axis=1, errors="ignore")
    y = data["plagiarized"]
    
    clf = DecisionTreeClassifier()

    # Make folds of data for cross validation
    k_fold = KFold(n_splits = 10, shuffle = True)

    results = []
    for train, test in k_fold.split(data):
        clf.fit(X.iloc[train], y.iloc[train])
        results.append(f1_score(y.iloc[test], clf.predict(X.iloc[test]), average = None))
    
    return sum(results) / len(results)

p = np.linspace(0.8, 3, num = 50)
F1 = np.stack(list(map(calc_F1_prop, p.tolist())), axis = 0)
# Filter F1
filtered_F1 = np.column_stack((lowess(F1[:, 0], p, is_sorted=True, frac=0.2, it=0)[:, 1],
                             lowess(F1[:, 1], p, is_sorted=True, frac=0.2, it=0)[:, 1]))

# Normal plot
prop_plot = figure(title = "F1 score as a function of the proportion of texts in the data",
                   x_axis_label = "Relative proportion of non-plagiared to plagiarized",
                   y_axis_label = "F1")
prop_plot.line(p, F1[:, 0], legend_label = "Non-plagiarized", line_color = "steelblue")
prop_plot.line(p, F1[:, 1], legend_label = "Plagiarized", line_color = "orange")


# Filtered plot to remove some noise and better appreciate tendencies
prop_filtered_plot = figure(title = "F1 score as a function of the proportion of texts in the data",
                           x_axis_label = "Relative proportion of non-plagiared to plagiarized",
                           y_axis_label = "F1")

prop_filtered_plot.line(p, filtered_F1[:, 0], legend_label = "Non-plagiarized", line_color = "steelblue")
prop_filtered_plot.line(p, filtered_F1[:, 1], legend_label = "Plagiarized", line_color = "orange")


show(gridplot([[prop_plot, prop_filtered_plot]], sizing_mode='scale_both'))


# The optimum point for the unbalanced data is obtained when both sets have the same proportion

Como podemos observar, el compromiso optimo para ambas clases en cuanto a precisión se encuentra en p = 1, es decir, la misma cantidad de casos de plagio que de no plagio.

## Optimización del número mínimo de muestras por hoja

En este apartado estuadiamos como varia el puntuador según la cantidad de muestras minimas que se necesitan en cada nueva hoja
para poder formar un nuevo nodo.

In [9]:
def calc_F1_leaf(leafs):
    clf = RandomForestClassifier(min_samples_leaf = leafs, max_depth = 7)

    # Make folds of data for cross validation
    k_fold = KFold(n_splits = 10, shuffle = True)

    results = []
    for train, test in k_fold.split(X):
        clf.fit(X.iloc[train], y.iloc[train])
        results.append(f1_score(y.iloc[test], clf.predict(X.iloc[test]), average = None))
    
    return sum(results) / len(results)
        
leafs = np.arange(1, 30, 1)
F1 = np.stack(list(map(calc_F1_leaf, leafs)), axis = 0)
filtered_F1 = np.column_stack((lowess(F1[:, 0], leafs, is_sorted=True, frac=0.06, it=0)[:, 1],
                             lowess(F1[:, 1], leafs, is_sorted=True, frac=0.06, it=0)[:, 1]))
# Normal plot
min_samples_plot = figure(title = "F1 score as a function of the minimum samples per leaf",
                   x_axis_label = "Minimum samples per leaf",
                   y_axis_label = "F1")

min_samples_plot.line(leafs, F1[:, 0], legend_label = "Non-plagiarized", line_color = "steelblue")
min_samples_plot.line(leafs, F1[:, 1], legend_label = "Plagiarized", line_color = "orange")


# Filtered plot to remove some noise and better appreciate tendencies
min_samples_plot_filtered = figure(title = "F1 score as a function of the minimum samples per leaf",
                           x_axis_label = "Minimum samples per leaf",
                           y_axis_label = "F1")

min_samples_plot_filtered.line(leafs, filtered_F1[:, 0], legend_label = "Non-plagiarized", line_color = "steelblue")
min_samples_plot_filtered.line(leafs, filtered_F1[:, 1], legend_label = "Plagiarized", line_color = "orange")


show(gridplot([[min_samples_plot, min_samples_plot_filtered]], sizing_mode='scale_both'))

Dado que el plot normal tiene mucho ruido, nos fijaremos en el suvizado por loess. Se puede ver una región en la que crece desde cero y la puntuación de los no plagiados cae rapidamente.

## Optimización del minimo número de muestras por división

Ahora estudiaremos como afecta a la precisión el minimo numero de muestras necesario para dividir un nodo en sus hijos. En este caso lo estudiaremos como la fracción del total de las muestras que le hemos pasado al clasificador.

In [27]:
def calc_F1_split(split):
    clf = RandomForestClassifier(min_samples_split = split, max_depth = 7, min_samples_leaf = 5)

    # Make folds of data for cross validation
    k_fold = KFold(n_splits = 10, shuffle = True)

    results = []
    for train, test in k_fold.split(X):
        clf.fit(X.iloc[train], y.iloc[train])
        results.append(f1_score(y.iloc[test], clf.predict(X.iloc[test]), average = None))
    
    return sum(results) / len(results)
        
splits = np.linspace(0.0001, 0.4, num = 30)
F1 = np.stack(list(map(calc_F1_split, splits)), axis = 0)
filtered_F1 = np.column_stack((lowess(F1[:, 0], splits, is_sorted=True, frac=0.06, it=0)[:, 1],
                               lowess(F1[:, 1], splits, is_sorted=True, frac=0.06, it=0)[:, 1]))
# Normal plot
split_plot = figure(title = "F1 score as a function of the fraction of the minimum samples required to split",
                   x_axis_label = "fraction",
                   y_axis_label = "F1")

split_plot.line(splits, F1[:, 0], legend_label = "Non-plagiarized", line_color = "steelblue")
split_plot.line(splits, F1[:, 1], legend_label = "Plagiarized", line_color = "orange")


# Filtered plot to remove some noise and better appreciate tendencies
split_plot_filtered = figure(title = "F1 score as a function of the fraction of minimum samples required to split",
                            x_axis_label = "fraction",
                            y_axis_label = "F1")

split_plot_filtered.line(splits, filtered_F1[:, 0], legend_label = "Non-plagiarized", line_color = "steelblue")
split_plot_filtered.line(splits, filtered_F1[:, 1], legend_label = "Plagiarized", line_color = "orange")


show(gridplot([[split_plot, split_plot_filtered]], sizing_mode='scale_both'))



## Optimización del desdeco minimo en la impureza

La impureza de un nodo mide a groso modo la homegenidad de de las etiquetas que tiene dicho nodo. El parametro que 
estamos estudiando lo que hace es ajustar si creara un nueva división en base a si la reducción en la impureza que supone es 
igual o menor a un valor establecido.

In [8]:
def calc_F1_impurity(imp):
    clf = RandomForestClassifier(min_impurity_decrease = imp)

    # Make folds of data for cross validation
    k_fold = KFold(n_splits = 10, shuffle = True)

    results = []
    for train, test in k_fold.split(X):
        clf.fit(X.iloc[train], y.iloc[train])
        results.append(f1_score(y.iloc[test], clf.predict(X.iloc[test]), average = None))
    
    return sum(results) / len(results)
        
imp = np.linspace(0, 1, num = 30)
F1 = np.stack(list(map(calc_F1_impurity, imp)), axis = 0)
filtered_F1 = np.column_stack((lowess(F1[:, 0], imp, is_sorted=True, frac=0.1, it=0)[:, 1],
                               lowess(F1[:, 1], imp, is_sorted=True, frac=0.1, it=0)[:, 1]))
# Normal plot
imp_plot = figure(title = "F1 score as a function of the minimum entropy decrease required to split",
                   x_axis_label = "fraction",
                   y_axis_label = "F1")

imp_plot.line(imp, F1[:, 0], legend_label = "Non-plagiarized", line_color = "steelblue")
imp_plot.line(imp, F1[:, 1], legend_label = "Plagiarized", line_color = "orange")


# Filtered plot to remove some noise and better appreciate tendencies
imp_plot_filtered = figure(title = "F1 score as a function of the minimum entropy decrease required to split",
                            x_axis_label = "fraction",
                            y_axis_label = "F1")

imp_plot_filtered.line(imp, filtered_F1[:, 0], legend_label = "Non-plagiarized", line_color = "steelblue")
imp_plot_filtered.line(imp, filtered_F1[:, 1], legend_label = "Plagiarized", line_color = "orange")


show(gridplot([[imp_plot, imp_plot_filtered]], sizing_mode='scale_both'))

# Any value that isn't zero will make our score drop quickly, no too interesting

Como podemos observar, a partir de 0.2 cae de forma drastica y oscila para ambas categorias. Por lo tanto lo dejaremos como nulo.

## Evalución del modelo depurado

In [ ]:
# Reevaluate the model with the slight optimizations found earlier
# Igual hay que hacerlo dinamicamente
clf = RandomForestClassifier(max_depth = 7, min_samples_split = 0.04, min_samples_leaf = 10)
eval_model_table(X, y, clf)

Como podemos ver, hemos mejorado (muy ligeramente) el modelo, llegando a tener un precisión cercana al 70%. Finalmente, guardamos el modelo para poder ser reusado posteriormente.

In [ ]:
import pickle
# We improved a bit, lets save it and work with this one
with open("../models/ngram_tok.model", "wb") as f:
    pickle.dump(clf, f)

In [ ]:
clf.predict(X)

In [ ]:
clf.predict_proba(X)